In [172]:
import tensorflow as tf
import numpy as np
import scipy as sp
import scipy.stats as stats
import multiprocessing as mp
import itertools

In [173]:
N = stats.norm.cdf

def BlackScholesCall(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T)* N(d2)

In [229]:
x = tf.constant(50.0)

with tf.GradientTape() as g:
    g.watch(x)
    C = BlackScholesCall(x,100,1,0,0.1)
    delta = g.gradient(C,x)

print(delta)

tf.Tensor(0.0, shape=(), dtype=float32)


In [195]:
def test(it, S):
    sigma = 0.1
    sum = 0.0
    for i in range(it):
        Z = np.random.normal()
        stock_price = S*np.exp((-0.5*sigma**2)+sigma*Z)
        if stock_price - 100 > 0:
            sum = sum + (stock_price - 100)
    
    return sum/float(it)
            

In [224]:
def test2(it, S):
    sigma = 0.1
    sum = tf.constant(0.0)
    stock_price = np.zeros(it)
    T = 1
    weeks = 52
    dt = T/weeks
    for i in range(it):
        Sold = S
        for t in range(weeks):
            Z = np.random.normal()
            Snew = Sold*np.exp((-0.5*sigma**2)*dt+sigma*np.sqrt(dt)*Z)
            Sold = Snew
        if Snew - 100 > 0:
            sum = sum + (Snew - 100)
    
    return sum/float(it)

In [196]:
print(test(100000,100))
print(BlackScholesCall(100,100,1,0,0.1))

4.002366711087956
3.987761167674492


In [207]:
x = tf.constant(100.0)

with tf.GradientTape() as g:
    g.watch(x)
    C = test(100000,x)
    delta = g.gradient(C,x)

print(delta)

tf.Tensor(0.52046776, shape=(), dtype=float32)


In [231]:
x = tf.constant(50.0)

with tf.GradientTape() as g:
    g.watch(x)
    C = test2(1000,x)
    delta = g.gradient(C,x)

print(delta)

None
